## Building a symbolic classifier

In [1]:
import skmine

print("This tutorial was tested with the following version of skmine :", skmine.__version__)

This tutorial was tested with the following version of skmine : 0.0.9


MDL based algorithms encode data according to a given codetable

When calling ``.fit``, we iteratively look for the codetable that compress
the training data the best

**When we are done with training our model, we can benefit from the refined codetable 
to make some predictions**

### SLIM Classifier

An **integrated classifier in scikit mine is available** and allows to **solve binary and multiclass problems**. It uses the SLIM compression algorithm. 

To use it, we need to have **discretized dataset**. Let's take for example the **discretized iris dataset**.

In [2]:
from skmine.datasets.fimi import fetch_iris
X, y = fetch_iris(return_y=True)  # without return_y=True, the method would have returned the whole dataset in one variable
print("X shape:", X.shape)
print("y shape:", y.shape)
X.head()

X shape: (150,)
y shape: (150,)


0     [2, 9, 12, 15]
1    [1, 10, 11, 14]
2    [5, 10, 13, 16]
3     [2, 6, 12, 15]
4     [1, 8, 11, 14]
Name: iris.D19.N150.C3.num, dtype: object

In [3]:
y.head()

0    18
1    17
2    19
3    18
4    17
Name: iris.D19.N150.C3.num, dtype: int64

The purpose of this dataset is to **predict the last column of db from the other 4**. The possible targets are: 17, 18, 19. We can prepare our train and test data set.

In [4]:
from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test) = train_test_split(X, y, random_state=1, test_size=0.2, shuffle=True)
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

X_train shape: (120,) y_train shape: (120,)
X_test shape: (30,) y_test shape: (30,)


Now we can use our **SlimClassifier**.

In [5]:
from skmine.itemsets.slim_classifier import SlimClassifier

# You can pass in parameter of your classifier the set of your items. 
# This will improve its performance especially on small data sets like iris.
items = set(item for transaction in X for item in transaction)

clf = SlimClassifier(items=items)  # You can also enable or disable the pruning of SLIM compressors via the `pruning` parameter
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8333333333333334

You can use many functions of sckit learn that are compatible with classifiers. For example, build a **confusion matrix**, use **GridSearchCV** or **cross validation**.

- **Confusion matrix**

In [6]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[13,  1,  0],
       [ 0,  8,  1],
       [ 0,  3,  4]])

- **GridSearchCV** (this method allows us to test many parameters for a classifier and to retain the best combination)

In [7]:
from sklearn.model_selection import GridSearchCV

parameters = {'pruning': [False, True], 'items': [None, items]}
grid = GridSearchCV(clf, parameters)
print(grid.fit(X_train,y_train))
print(grid.best_params_)
print(grid.score(X_train, y_train))

GridSearchCV(estimator=SlimClassifier(items={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                             12, 13, 14, 15, 16}),
             param_grid={'items': [None,
                                   {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                    14, 15, 16}],
                         'pruning': [False, True]})
{'items': {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}, 'pruning': False}
0.9833333333333333


With GridSearchCV we get with the best parameters an **accuracy of more than 98%**, much better than the previous score. With this combination, the item list is passed as a parameter and pruning is disabled. Since pruning does not improve the compression of codetables in the SLIM algorithm on iris, it does not matter whether it is enabled or not.

To be more sure of the quality of our results, we can use the cross validation of sklearn.
- **Cross validation**

In [8]:
from sklearn.model_selection import cross_val_score

cross_validation = cross_val_score(clf, X, y, cv=10)
print(cross_validation)
cross_validation.mean()

[0.93333333 0.93333333 0.86666667 0.93333333 0.93333333 0.93333333
 1.         1.         1.         0.93333333]


0.9466666666666667

After cross validation, we see that the **accuracy is almost 95% on average**. So in 95% of the cases, the right type of flower is given.

----------

### OneVsRest classifier

The **SLIM algorithm** is also compatible with **scikit-learn** to be used from other classifiers like **One-vs-the-rest (OvR)** (https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html). The limitation of this method is that the classifier **only works for multiclass classification** problems while the embedded classifier works for both binary and multiclass problems.

In [9]:
import pandas as pd
from skmine.itemsets import SLIM
from sklearn.preprocessing import MultiLabelBinarizer

In [10]:
class TransactionEncoder(MultiLabelBinarizer):  # pandas DataFrames are easier to read ;)
    def transform(self, X):
        _X = super().transform(X)
        return pd.DataFrame(data=_X, columns=self.classes_)

In [11]:
transactions = [ 
     ['bananas', 'milk'], 
     ['milk', 'bananas', 'cookies'], 
     ['cookies', 'butter', 'tea'], 
     ['tea'],  
     ['milk', 'bananas', 'tea'], 
]
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D 

,bananas,butter,cookies,milk,tea
0,1,0,0,1,0
1,1,0,1,1,0
2,0,1,1,0,1
3,0,0,0,0,1
4,1,0,0,1,1


In [12]:
slim = SLIM()
slim.fit(D).transform(D)

,itemset,usage
0,"[bananas, milk]",3
1,[tea],3
2,[cookies],2
3,[butter],1


We keep this **codetable** in mind, as we will later use it **to interpret our predictions**

----------
#### First "predictions" 

We define a new transactional dataset, and can call our ``decision_function`` on it. This will yield ``distances`` w.r.t the encoding scheme provided by our codetable

In [13]:
new_transactions = [ 
   ['bananas', 'milk'], 
   ['milk', 'sirup', 'cookies'], 
   ['butter', 'tea'], 
   ['tea'],  
   ['milk', 'bananas', 'tea'], 
]
new_D = te.transform(new_transactions)
new_D

/home/cregan/fsl/lib/python3.10/site-packages/scikit_learn-1.2.0-py3.10-linux-x86_64.egg/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['sirup'] will be ignored
  warnings.warn(


,bananas,butter,cookies,milk,tea
0,1,0,0,1,0
1,0,0,1,1,0
2,0,1,0,0,1
3,0,0,0,0,1
4,1,0,0,1,1


In [14]:
codes = slim.decision_function(new_D)

In [15]:
pd.DataFrame([pd.Series(new_transactions), codes], index=['transaction', 'distance']).T

,transaction,distance
0,"[bananas, milk]",-1.906891
1,"[milk, sirup, cookies]",-6.228819
2,"[butter, tea]",-4.813781
3,[tea],-1.906891
4,"[milk, bananas, tea]",-3.813781


#### Built-in interpretations
Now we can interpret codes for the new data, directly by **looking at the codetable inferred from training data**

First observations

* Entry 1 has the highest distance w.r.t the encoding scheme.
  You can see it contains `milk`, `sirup` and `cookies`. From the codetable we see `milk` and `cookies` are not    grouped together, while `sirup` has never been seen
  

*  Entry 4 has the lowest distance. It contains `bananas` and `milk`, which are grouped together in the codetable and have high occurence in the training data.

#### Shortest code wins !!
Next, we are going to use an ensemble of SLIM encoding schemes, and utilize them via a ``OneVsRest`` methodology, to perform **multi-class classification**.
The methodology is very simple

1. We clone our base estimator as many time as we need (one per class)
2. We fit every estimator on entries corresponding to its class in the input data
3. When calling ``.predict``, we actually call ``.decision_function`` and get distances from our decision boundaries for every class
4. The shorted code wins : we choose the class with the lowest distance for a given transaction

In [16]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline

In [17]:
pipe = Pipeline([
    ('transaction_encoder', TransactionEncoder(sparse_output=False)),
    ('slim', SLIM()),
])

In [18]:
transactions = [
    ['milk', 'bananas'],
    ['tea', 'New York Times', 'El Pais'],
    ['New York Times'],
    ['El Pais', 'The Economist'],
    ['milk', 'tea'],
    ['croissant', 'tea'],
    ['croissant', 'chocolatine', 'milk'],
]
y = [
    'foodstore', 
    'newspaper', 
    'newspaper', 
    'newspaper', 
    'foodstore',
    'bakery',
    'bakery',
]

In [19]:
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D

,El Pais,New York Times,The Economist,bananas,chocolatine,croissant,milk,tea
0,0,0,0,1,0,0,1,0
1,1,1,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1,1
5,0,0,0,0,0,1,0,1
6,0,0,0,0,1,1,1,0


In [20]:
ovr = OneVsRestClassifier(SLIM())

In [21]:
ovr.fit(D, y=y)
ovr.estimators_

[SLIM(), SLIM(), SLIM()]

In [22]:
pd.DataFrame(ovr.decision_function(D), columns=ovr.classes_)

,bakery,foodstore,newspaper
0,-7.169925,-4.584963,-7.614710
1,-10.754888,-9.754888,-7.252140
2,-3.584963,-3.584963,-2.222392
3,-7.169925,-7.169925,-5.029747
4,-7.169925,-4.584963,-6.614710
5,-2.584963,-6.169925,-6.614710
6,-2.584963,-9.169926,-11.422065


In [23]:
ovr.predict(D)

array(['foodstore', 'newspaper', 'newspaper', 'newspaper', 'foodstore',
       'bakery', 'bakery'], dtype='<U9')